In [ ]:
import os
import re
import pandas as pd
from ast import literal_eval
from difflib import SequenceMatcher
import numpy as np
from eunjeon import Mecab

mecab = Mecab()
df_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#9_tf_idf'}.xlsx")
df = pd.read_excel(df_name, index_col=0)
df[['case', 'law']] = df[['case','law']].fillna(value='[]')

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
cases = list(df["case"])    
cases = [item for sublist in cases for item in literal_eval(sublist)]
cases = list(set(cases))
cases = [c for c in cases if (len(c) > 2 and len(mecab.nouns(c)) > 1) or (len(c) <= 2 and len(mecab.nouns(c)) > 0)]
cases.sort()


sim =set()
for i in range(len(cases)):
    for j in range(i+1, len(cases)):
        if similar(cases[i], cases[j]) > 0.5:# 0.5
            sim.add(cases[j])
                
cases = list(set(cases) - sim)
cases.sort()
result = pd.DataFrame(0, index=np.arange(len(df)), columns = cases)
result['cid'] = df['cid']
cols = result.columns.tolist()
cols = cols[-1:] + cols[0:-1]
result = result[cols]

In [ ]:
for i in range(len(df['case'])):
    
    cs = literal_eval(df.loc[i, 'case'])
    for c in cs:
        for j in range(len(cases)):
            if similar(c, cases[j]) > 0.5:# 0.5
                result.at[i, cases[j]] = 1

In [ ]:
count = 0
new_cases = list()
for i in range(len(cases)):
    if len(result[result[cases[i]] == 1]) < 4: #5
        count+=1
    else:
        new_cases.append(cases[i])

In [ ]:
result = pd.DataFrame(0, index=np.arange(len(df)), columns = new_cases)
result['cid'] = df['cid']
cols = result.columns.tolist()
cols = cols[-1:] + cols[0:-1]
result = result[cols]

for i in range(len(df['case'])):
    
    cs = literal_eval(df.loc[i, 'case'])
    for c in cs:
        for j in range(len(new_cases)):
            if similar(c, new_cases[j]) > 0.5:# 0.8
                result.at[i, new_cases[j]] = 1


x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_case'}.xlsx")
result.to_excel(x2name)

In [ ]:
laws = list(df['law'])    
laws = [item.replace('\n', '').replace(' ', '') for sublist in laws for item in literal_eval(sublist)]
laws = list(set(laws))
laws.sort()
result = pd.DataFrame(0, index=np.arange(len(df)), columns = laws)
result['cid'] = df['cid']
cols = result.columns.tolist()
cols = cols[-1:] + cols[0:-1]
result = result[cols]

In [ ]:
for i in range(len(df['law'])):
    
    lw = literal_eval(df.loc[i, 'law'])
    for l in lw:
        for j in range(len(laws)):
            if l.replace('\n', '').replace(' ', '') == laws[j]:
                result.at[i, laws[j]] = 1

In [ ]:
count = 0
new_law = list()
for i in range(len(laws)):
    if len(result[result[laws[i]] == 1]) < 4: #3
        count+=1
    else:
        new_law.append(laws[i])

In [ ]:
result = pd.DataFrame(0, index=np.arange(len(df)), columns = new_law)
result['cid'] = df['cid']
cols = result.columns.tolist()
cols = cols[-1:] + cols[0:-1]
result = result[cols]


for i in range(len(df['law'])):
    
    lw = literal_eval(df.loc[i, 'law'])
    for l in lw:
        for j in range(len(new_law)):
            if l.replace('\n', '').replace(' ', '') == new_law[j]:
                result.at[i, new_law[j]] = 1
                
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_law'}.xlsx")
result.to_excel(x2name)

In [ ]:
len(new_law), len(new_cases)

In [ ]:
df = df.drop(columns = ['case', 'law'])
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_final'}.xlsx")
df.to_excel(x2name)

In [ ]:
df_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_case'}.xlsx")
df = pd.read_excel(df_name, index_col=0)
print(list(df.columns))

In [ ]:
df_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_law'}.xlsx")
df = pd.read_excel(df_name, index_col=0)
print(list(df.columns))